In [1]:
import tpx
import numpy

In [2]:
# Select a file to load:
fname = "/home/cadams/CRAB/UTA-tpx3/120thresh_12biasvolt_IIgain0V000000.tpx3"

In [3]:
#Load the file from the binary data:
crab0_data = tpx.frame(fname=fname)

## Slicing frames into events.

The goal of this notebook is to develop the function that will take an entire frame of unbuffered events and, without S1, find regions of interest and remove background events.

First, let's look at the time of hits:

In [4]:
hit_times = crab0_data["TOA"]
hit_tot   = crab0_data["TOT"]

# hit times are returned in seconds from the data reader, resolution is in ns:
print(hit_times[0:100])

[0.06795924 0.06795927 0.07022654 0.07159322 0.07159381 0.07159388
 0.07159399 0.07159332 0.07159416 0.07159439 0.07159443 0.07159484
 0.07159521 0.07159599 0.07159609 0.07159537 0.07159493 0.07159652
 0.07159415 0.07159659 0.07159382 0.07159483 0.07159442 0.0715969
 0.07159543 0.07159489 0.07159519 0.07159517 0.07159519 0.0716634
 0.0716638  0.07166489 0.07166573 0.07166583 0.08321898 0.08322167
 0.08322153 0.08322398 0.08322418 0.09175141 0.09175299 0.09175295
 0.09175379 0.09175389 0.09175381 0.09175392 0.0917538  0.09175384
 0.09175468 0.09175533 0.09175483 0.09175294 0.09175535 0.09175533
 0.09175292 0.09175478 0.09175463 0.09175539 0.09175533 0.09175513
 0.09175373 0.09175533 0.0917552  0.09175293 0.09175516 0.09175533
 0.1049433  0.10914181 0.10975783 0.13738    0.14101739 0.14101734
 0.15865706 0.15865703 0.16379254 0.16379321 0.16379357 0.1637943
 0.16379418 0.16961621 0.16961639 0.16961653 0.1696172  0.16961691
 0.1696176  0.1696171  0.1696172  0.1696176  0.16961758 0.1696175

In [5]:
bin_size = 100*1e-6 # microseconds
hit_bins = numpy.arange(0,0.5, bin_size)
print(hit_bins.shape)

(5000,)


In [24]:
def bin_tpx_data_for_slicing(tpx_frame, bin_size=100e-6, threshold=2, method="toa_counts"):
    """Bin the data to accomodate event slicing.  Parameters:

    Args:
        tpx_frame (tpx.Frame): tpx.Frame object to be sliced
        bin_size (_type_): total bin size in seconds (ie, 1 us would be 1e-6)
        threshold (_type_): Threshold to apply to each bin
        method (str, optional): method of slicing, either "toa_counts" or "tot_sum". Defaults to "toa_counts".
    """
    
    # First, create a list of bins:

    max_time = numpy.max(tpx_frame["TOA"])
    max_time = 0.5

    time_bins = numpy.arange(0,max_time, bin_size)

    # Bin all the hit times into the bins to determine how much activity was in each bin:
    binned_activity, bin_edges = numpy.histogram(hit_times, bins=time_bins)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])

    # Apply the bin thresholds:

    if method == "toa_counts":
        non_zero_bins   = binned_activity >= threshold
        non_zero_times  = bin_centers[non_zero_bins]
        non_zero_counts = binned_activity[non_zero_bins]
    elif method == "tot_sum":
        # To do this based on total ToT per bin is trickier and slower but not really slow:
        tot_per_bin = numpy.asarray([
            numpy.sum(hit_tot[ numpy.where((hit_times > low) & (hit_times <= high)) ])
            for low, high in zip(time_bins[:-1], time_bins[1:])
        ])
        non_zero_bins   = tot_per_bin >= tot_threshold
        non_zero_times  = bin_centers[non_zero_bins]
        non_zero_counts = tot_per_bin[non_zero_bins]

    return time_bins, numpy.where(non_zero_bins)[0], non_zero_times, non_zero_counts

In [25]:
time_bins, non_zero_bins, non_zero_times, non_zero_counts = bin_tpx_data_for_slicing(crab0_data)

In [26]:
print(time_bins.shape)
print(non_zero_bins)
print(non_zero_times.shape)
print(non_zero_counts.shape)

(5000,)
[  34   56   67   76   98  135  186  239  258  336  353  369  395  412
  440  469  474  546  547  555  629  638  679  687  691  715  716  758
  777  832  895  897  903  917  919 1038 1072 1106 1109 1113 1159 1243
 1257 1271 1297 1375 1410 1464 1518 1562 1586 1598 1637 1696 1711 1742
 1754 1768 1769 1853 1857 1860 1872 1915 1929 2009 2037 2048 2069 2085
 2156 2172 2205 2217 2243 2268 2354 2366 2385 2476 2477 2532 2592 2608
 2642 2661 2722 2745 2773 2785 2881 2897 2904 2911 2989 3009 3046 3099
 3146 3193 3203 3222 3368 3390 3395 3402 3409 3487 3525 3530 3538 3569
 3575 3641 3648 3658 3699 3707 3764 3792 3809 3966 3995 4098 4128 4137
 4139 4149 4154 4160 4165 4166 4283 4314 4355 4438 4439 4481 4559 4569
 4592 4637 4733 4861 4865 4871 4884 4921 4954 4973]
(150,)
(150,)


In [79]:
def select_regions_of_interest(non_zero_bins, original_bins):
    """Take the list of bins and return the regions of interest in min/max.

    Args:
        non_zero_bins (numpy.ndarray[int]): List of histogram bins from a frame that are not zero
        original_bins (numpy.ndarray): The original histogram's bins

    Returns:
        _type_: _description_
    """


    bin_spacing = non_zero_bins[1:] - non_zero_bins[:-1]
    # this variable says whether the bin at [i+1] is directly adjacent to [i]

    temp_adjacent_bins = bin_spacing == 1
    # in order to select original bin indexes from this, pad it with a False:

    adjacent_bins = numpy.zeros_like(non_zero_bins, dtype="bool")
    # And then add the originals in:
    adjacent_bins[1:] = temp_adjacent_bins

    # The easiest way to vectorize this is the following:
    # For the list of non-adjacent bins, we just select the boundaries from the original histogram bins.
    # For the list of adjacent bins, we extend the bins in the selected bins

    # We never start a region of interest on a bin that has an adjacency
    starting_indexes = non_zero_bins[adjacent_bins != 1]
    ending_indexes   = starting_indexes + 1

    extend_bins = non_zero_bins[adjacent_bins == 1]

    # For each bin in the "extend bins" category, we find the matching end bin and increment it:
    for bin in extend_bins:
        index = numpy.where(ending_indexes == bin)[0]
        ending_indexes[index] += 1

    # TODO: The above loop does not work if there are multiple consecutive bins to merge!


    # Finally, take the bin indexes and use them in the original bins to get upper and lower bounds:
    lower_bounds = original_bins[starting_indexes]
    upper_bounds = original_bins[ending_indexes]

    return lower_bounds, upper_bounds


In [81]:
lower_bounds, upper_bounds = select_regions_of_interest(non_zero_bins, time_bins)

In [82]:
def slice_frame_into_events(tpx_frame, lower_bounds, upper_bounds):
    """Time-slice the 

    Args:
        tpx_frame (tpx.frame): original frame of events
        lower_bounds (numpy.ndarray): Array of times, in [seconds], for the upper bounds
        upper_bounds (numpy.ndarray): Array of times, in [seconds], for the lower bounds

    Returns:
        List[tpx.frame]: List of frames sliced by the suggested bounds
    """

    events = [
        tpx_frame.time_slice(lower, upper) for lower, upper in zip(lower_bounds, upper_bounds)
    ]

    return events

In [86]:
events = slice_frame_into_events(crab0_data, lower_bounds, upper_bounds)

In [85]:
len(events)

144

In [87]:
print(events[0])